### Executar apenas no GOOGLE COLAB

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 60.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=c9dbc20589d3ab0c643828d61ad19b0f4314c11abc4d2e092a80e4aeb64382c3
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


### Cria um mount point no seu Google Drive para acessar os dados do DataLake

- Execute e passe o código solicitado

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from pyspark.sql import SparkSession

Inicialize uma Sessão do Spark


In [ ]:
spark = SparkSession.builder.appName('desafio_a3').getOrCreate()

Criando Dataframe df_commerce

In [ ]:
df_commerce = spark.read.format('json')\
 .load('/content/drive/MyDrive/datalake/datalake/landing-zone/commerce/commerce_*.json')

In [ ]:
df_commerce.count()

800

In [ ]:
df_commerce.printSchema()

root
 |-- department: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)



In [ ]:
df_commerce = df_commerce.select('department','product_name', 'price','user_id')

In [ ]:
df_commerce.write.mode('overwrite').format('parquet')\
  .save('/content/drive/MyDrive/datalake/datalake/processing-zone/commerce')

Criando Dataframe df_device

In [ ]:
df_device = spark.read.format('json')\
  .load('/content/drive/MyDrive/datalake/datalake/landing-zone/device/device_*.json')

In [ ]:
df_device.count()

800

In [ ]:
df_device.printSchema()

root
 |-- platform: string (nullable = true)
 |-- model: string (nullable = true)
 |-- user_id: long (nullable = true)



In [ ]:
df_device = df_device.select('platform','model','user_id')

In [ ]:
df_device.write.mode('overwrite').format('parquet')\
  .save('/content/drive/MyDrive/datalake/datalake/processing-zone/device')

Criando Dataframe df_user

In [ ]:
df_user = spark.read.format('json')\
  .load('/content/drive/MyDrive/datalake/datalake/landing-zone/user/user_*.json')

In [ ]:
df_user.count()

800

In [ ]:
df_user.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- date_of_birth: string (nullable = true)



In [ ]:
df_user = df_user.select('user_id','first_name','last_name','email','date_of_birth')

In [ ]:
df_user.write.mode('overwrite').format('parquet')\
  .save('/content/drive/MyDrive/datalake/datalake/processing-zone/user')

Lendo Arquivos Parquet

In [ ]:
df_commerce_pro = spark.read.format('parquet')\
  .load('/content/drive/MyDrive/datalake/datalake/processing-zone/commerce/*.parquet')

In [ ]:
df_device_pro = spark.read.format('parquet')\
  .load('/content/drive/MyDrive/datalake/datalake/processing-zone/device/*.parquet')

In [ ]:
df_user_pro = spark.read.format('parquet')\
  .load('/content/drive/MyDrive/datalake/datalake/processing-zone/user/*.parquet')

Unindo df_commerce e df_device

In [ ]:
df_commerce_device = df_commerce_pro.select('department','product_name', 'price','user_id')\
  .join(df_device_pro.select('platform','model','user_id'), ['user_id'],"left")

In [ ]:
df_commerce_device.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- department: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- platform: string (nullable = true)
 |-- model: string (nullable = true)



Unindo df_commerce_device e df_user

In [ ]:
df_commerce_device_user = df_commerce_device\
  .join(df_user_pro.select('user_id','first_name','last_name','email','date_of_birth'), ['user_id'],"left")

In [ ]:
df_commerce_device_user.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- department: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- platform: string (nullable = true)
 |-- model: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- date_of_birth: string (nullable = true)



Gravando df_commerce_device_user em consume-zone

In [ ]:
df_commerce_device_user.write.mode('overwrite').format('parquet')\
  .save('/content/drive/MyDrive/datalake/datalake/consumer-zone/commerce_device_user')

In [ ]:
df_commerce_device_user.show()

+-------+--------------------+--------------------+-----+----------+----------+----------+----------+--------------------+-------------+
|user_id|          department|        product_name|price|  platform|     model|first_name| last_name|               email|date_of_birth|
+-------+--------------------+--------------------+-----+----------+----------+----------+----------+--------------------+-------------+
|   1056|        Baby & Tools|Lightweight Woode...|46.07|      null|      null|      null|      null|                null|         null|
|   1568|      Health & Games|Mediocre Concrete...| 2.83|      null|      null|      null|      null|                null|         null|
|   3272|               Music|Aerodynamic Paper...|80.08|      null|      null|    Dudley|McCullough|dudley.mccullough...|   1975-05-10|
|   3473|         Electronics|Practical Plastic...|87.25|      null|      null|      null|      null|                null|         null|
|   4207|              Movies|Small Concr